# Linear Regression Example Project
You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.
    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [ ]:
from pyspark.sql import *
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [1]:
# Use Spark to read in the Ecommerce Customers csv file.
df = sqlContext.read.format("com.databricks.spark.csv").options(header='true').options(inferSchema='true').load("file:/cruise_ship_info.csv")


In [2]:
# Print the Schema of the DataFrame
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [3]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [4]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [5]:
df.groupby('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|  Royal_Caribbean|   23|
|          Oceania|    3|
|            Costa|   11|
|           Disney|    2|
|        Celebrity|   10|
|           Cunard|    3|
|         Carnival|   22|
|              P&O|    6|
|          Crystal|    2|
|        Norwegian|   13|
|           Orient|    1|
|        Silversea|    4|
|             Star|    6|
|          Azamara|    2|
|         Windstar|    3|
|Regent_Seven_Seas|    5|
|         Princess|   17|
| Holland_American|   14|
|              MSC|    8|
|         Seabourn|    3|
+-----------------+-----+



In [6]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="crusie_cat")
indexed = indexer.fit(df).transform(df)

In [7]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(
    inputCols=['Age','Tonnage', 'passengers',  'length',
 'cabins', 'passenger_density', 'crusie_cat'],
    outputCol="features")

In [9]:
output = assembler.transform(indexed)
output.select("features").head(6)

[Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(features=DenseVector([26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0])),
 Row(features=DenseVector([11.0, 110.0, 29.74, 9.53, 14.88, 36.99, 1.0])),
 Row(features=DenseVector([17.0, 101.353, 26.42, 8.92, 13.21, 38.36, 1.0])),
 Row(features=DenseVector([22.0, 70.367, 20.52, 8.55, 10.2, 34.29, 1.0]))]

In [10]:
final_data = output.select("features",'crew')
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [11]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              109|
|   mean|7.566880733944954|
| stddev|3.348124045571309|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [12]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                49|
|   mean| 8.299795918367346|
| stddev|3.8140433978674557|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [14]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [15]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [16]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.013364880828,0.0177218722717,-0.168335065442,0.357204480556,0.858060047633,-0.0112391379498,0.0364231671648] Intercept: -0.364341500967


In [17]:
test_results = lrModel.transform(test_data)
test_results.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[5.0,86.0,21.04,9...|  8.0| 9.313878891667386|
|[6.0,93.0,23.94,9...|11.09|10.600716035502828|
|[8.0,110.0,29.74,...| 11.6|12.248915528853924|
|[9.0,110.0,29.74,...| 11.6|12.239122692831504|
|[10.0,68.0,10.8,7...| 6.36| 6.378324840481488|
|[10.0,105.0,27.2,...|10.68|11.346791733082863|
|[11.0,58.6,15.66,...|  7.6| 7.383816076245801|
|[11.0,86.0,21.24,...|  9.3| 9.571116360506032|
|[12.0,2.329,0.94,...|  0.6|0.7418046565767158|
|[12.0,88.5,21.24,...|  9.3|10.446854025210087|
|[13.0,61.0,13.8,7...|  6.0| 6.522073033914489|
|[13.0,85.619,21.1...|  9.2|  9.62713247764695|
|[14.0,63.0,14.4,7...| 5.61| 6.712071551268167|
|[16.0,77.71300000...| 9.09| 8.501701578050067|
|[17.0,75.166,19.2...| 7.66| 8.468291519764797|
|[18.0,70.60600000...| 8.58| 7.856282222920021|
|[20.0,53.049,13.4...|  6.0| 6.181235738591412|
|[20.0,55.451,12.6...| 5.88| 5.836750069

/usr/lib/spark/python/pyspark/ml/regression.py:123: UserWarning: weights is deprecated. Use coefficients instead.
  warnings.warn("weights is deprecated. Use coefficients instead.")


In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# evaluate RMSE
from math import sqrt
def calcRMSE(df, labelCol='crew'):
  """Given a DataFrame with 'label' and 'prediction' columns, calculate the root-mean-squared error
    Args:
        df: a DataFrame with labels and predictions        
    Returns:
        Float
  """
  return sqrt(float(df
                     .rdd
                     .map(lambda row: (row[labelCol]-row['prediction'])**2)
                     .reduce(lambda a,b: a+b)
                    ) / df.count() )

# evaluate R-squared
def calcRsq(df, labelCol='crew'):
  """Given a DataFrame with 'label' and 'prediction' columns, calculate the R-squared diagnostic  
    Args:
        df: a DataFrame with labels and predictions        
    Returns:
        Float
  """
  npts = df.count()
  mean_label = float(df.rdd.map(lambda row: row[labelCol]).reduce(lambda a,b: a+b))/npts
  total_squared_error = (df
                        .rdd
                        .map(lambda row: (row[labelCol]-mean_label)**2)
                        .reduce(lambda a,b: a+b)
                        )
  squared_error = (df
                   .rdd
                   .map(lambda row: (row[labelCol]-row['prediction'])**2)
                   .reduce(lambda a,b: a+b)
                  )  
  return (1.0 - float(squared_error) / total_squared_error)

print(calcRMSE(test_results))
print(calcRsq(test_results))

0.826631899395
0.952047869996


In [19]:
from pyspark.sql.functions import corr

In [20]:
df.select(corr('crew', 'passengers')).show()

+-------------------------+
|corr(crew,passengers,0,0)|
+-------------------------+
|       0.9152341306065387|
+-------------------------+



In [21]:
df.select(corr('crew', 'cabins')).show()

+---------------------+
|corr(crew,cabins,0,0)|
+---------------------+
|   0.9508226063578502|
+---------------------+



## Great Job!